In [1]:
import cv2
from ultralytics import YOLO
import random


def process_video_with_tracking(model, input_video_path, show_video=True, save_video=False, output_video_path="soda_defects_mini"):
    # Open the input video file
    cap = cv2.VideoCapture(input_video_path)

    if not cap.isOpened():
        raise Exception("Error: Could not open video file.")

    # Get input video frame rate and dimensions
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define the output video writer
    if save_video:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        results = model.track(frame, iou=0.4, conf=0.8, persist=True, imgsz=640, verbose=False, tracker="botsort.yaml")

        if results[0].boxes.id != None: # this will ensure that id is not None -> exist tracks
            boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
            ids = results[0].boxes.id.cpu().numpy().astype(int)

            for box, id in zip(boxes, ids):
                # Generate a random color for each object based on its ID
                random.seed(int(id))
                color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
                
                cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3],), color, 2)
                cv2.putText(
                    frame,
                    f"Id {id}",
                    (box[0], box[1]),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0, 255, 255),
                    2,
                )
        
        if save_video:
            out.write(frame)

        if show_video:
            frame = cv2.resize(frame, (0, 0), fx=0.75, fy=0.75)
            cv2.imshow("frame", frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    # Release the input video capture and output video writer
    cap.release()
    if save_video:
        out.release()

    # Close all OpenCV windows
    cv2.destroyAllWindows()

# Example usage:
model = YOLO('runs/detect/train/weights/best.pt')
model.fuse()
process_video_with_tracking(model, "soda_defects_mini.mp4", show_video=True, save_video=False, output_video_path="soda_defects_mini.mp4")



Model summary (fused): 218 layers, 25844392 parameters, 0 gradients, 78.7 GFLOPs


KeyboardInterrupt: 

In [ ]:
import cv2
from ultralytics import YOLO
import random


def draw_bounding_boxes_without_id(frame, results):
    boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
    classes = results[0].boxes.cls.cpu().numpy().astype(int)

    for box, clss in zip(boxes, classes):
        # Generate a random color for each object based on its ID
            random.seed(int(clss)+8)
            color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
            
            cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3],), color, 2)
            cv2.putText(
                frame,
                f"{model.model.names[clss]}",
                (box[0], box[1]),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (50, 255, 50),
                2,
            )
    return frame


def process_video_with_tracking(model, input_video_path, show_video=True, save_video=False, output_video_path="soda_defects_mini.mp4"):
    # Open the input video file
    cap = cv2.VideoCapture(input_video_path)

    if not cap.isOpened():
        raise Exception("Error: Could not open video file.")

    # Get input video frame rate and dimensions
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define the output video writer
    if save_video:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        results = model.track(frame, iou=0.4, conf=0.02, persist=True, imgsz=608, verbose=False, tracker="bytetrack.yaml", classes=8)
        results_detect = model_detect.predict(frame, iou=0.4, conf=0.02, imgsz=608, verbose=False)
    
        if results[0].boxes.id != None: # this will ensure that id is not None -> exist tracks
            boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
            ids = results[0].boxes.id.cpu().numpy().astype(int)

            for box, id in zip(boxes, ids):
                # Generate a random color for each object based on its ID
                random.seed(int(id))
                color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
                
                #cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3],), color, 2)
                #cv2.putText(
                #    frame,
                #    f"Id {id}",
                #    (box[0], box[1]),
                #    cv2.FONT_HERSHEY_SIMPLEX,
                #    0.70,
                #    (0, 255, 255),
                #    2,
                #)

        if results_detect[0].boxes != None:
            draw_bounding_boxes_without_id(frame, results_detect)
            
        
        if save_video:
            out.write(frame)

        if show_video:
            frame = cv2.resize(frame, (0, 0), fx=0.75, fy=0.75)
            cv2.imshow("frame", frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    # Release the input video capture and output video writer
    cap.release()
    if save_video:
        out.release()

    # Close all OpenCV windows
    cv2.destroyAllWindows()
    return results_detect, results

# Example usage:
model = YOLO('runs/detect/train/weights/best.pt')
model_detect = YOLO('runs/detect/train/weights/best.pt')
model.fuse()
model_detect.fuse()
results_detect, results = process_video_with_tracking(model, "soda_defects_mini.mp4", show_video=True, save_video=False, output_video_path="soda_defects_mini.mp4")



Model summary (fused): 218 layers, 25844392 parameters, 0 gradients, 78.7 GFLOPs
Model summary (fused): 218 layers, 25844392 parameters, 0 gradients, 78.7 GFLOPs
